In [ ]:
!pip install pdfplumber

In [ ]:
import pdfplumber

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pdfplumber")


In [ ]:
import pdfplumber

pdf_path='google_terms_of_service_en_in.pdf'

output_text_file='extracted_text.txt'

with pdfplumber.open(pdf_path) as pdf:
    extracted_text=""
    for page in pdf.pages:
        extracted_text+=page.extract_text()

with open(output_text_file, 'w') as text_file:
    text_file.write(extracted_text)

print(f"Text extracted and saved to {output_text_file}")

In [ ]:
# reading content

with open('extracted_text.txt', 'r') as file:
    document= file.read()

#preview content 
print(document[:500])  #first 500 characters

In [ ]:
# summarizing document

from transformers import pipeline

#loading summarization pipeline
summarizer = pipeline('summarization', model='t5-small')

summary = summarizer(document[:3000], max_length=200, min_length= 50, do_sample = False)

print(f'summary:{summary}')

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

#split text into sentences
sentences = sent_tokenize(document)

print(sentences[:10])

In [ ]:
# combine sentences into paragraphs

paragraph=[]
current_passage=''

for sentence in sentences:
    if len(current_passage.split())+len(sentence.split()) <200:            # adjusting the word limit
        current_passage+=' '+ sentence
    else:
        paragraph.append(current_passage.strip())                       # if word count more then adding in next passage
        current_passage= sentence

if current_passage:
    paragraph.append(current_passage.strip())                              # last passage included

In [ ]:
!pip install sentencepiece

In [ ]:
# Load the question generation pipeline
qg  = pipeline("text2text-generation", model="valhalla/t5-base-qg-hl")

# Function to generate 3 questions
def generate_questions_pipeline(paragraph, min_questions=3):
    input_text = paragraph
    results = qg(input_text)
    questions = results[0]['generated_text'].split('<sep>')

    # Ensure at least 3 questions by generating more if needed
    questions = [q.strip() for q in questions if q.strip()]
    
    while len(questions) < min_questions:
        extra_results = qg(input_text)
        extra_questions = extra_results[0]['generated_text'].split('<sep>')
        questions.extend([q.strip() for q in extra_questions if q.strip()])
        questions = list(set(questions))                                              # Remove duplicates
    
    return questions[:min_questions]                                                  # Return exactly 3 questions

get_question= generate_questions_pipeline(paragraph, min_questions=3)
print(get_question)

In [ ]:
# load the QA pipeline
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# function to track and answer only unique questions
def answer_unique_questions(paragraph, qa_pipeline):
    answered_questions = set()  # to store unique questions

    for idx, passage in enumerate(paragraph):
        questions = generate_questions_pipeline(paragraph)

        for question in questions:
            if question not in answered_questions:  # check if the question has already been answered
                answer = qa_pipeline({'question': question, 'context': paragraph})
                print(f"Q: {question}")
                print(f"A: {answer['answer']}\n")
                answered_questions.add(question)  # add the question to the set to avoid repetition
        print(f"{'='*50}\n")
              
answer_unique_questions(paragraph, qa_pipeline)